## Named Entity Recognition with spaCy

Named Entity Recognition (NER) is a subtask of information extraction that locates and classifies named entities in a text. The named entities could be organizations, persons, locations, times, quantities, monetary values, percentages, and so on.

### Install spaCy with llm dependencies
```bash
pip install -U pip setuptools wheel
pip install -U spacy
python -m spacy download en_core_web_trf
pip install spacy-llm
```

### Create Configuration File
Create a config file `config.cfg`:
```s
[nlp]
lang = "en"
pipeline = ["llm"]
batch_size = 128

[components]

[components.llm]
factory = "llm"

[components.llm.model]
@llm_models = "spacy.Dolly.v1"
name = "dolly-v2-3b"

[components.llm.task]
@llm_tasks = "spacy.NER.v2"
labels = PERSON,ORGANISATION,LOCATION
examples = null

[components.llm.task.normalizer]
@misc = "spacy.LowercaseNormalizer.v1"
```

### Run the code
```python
from pathlib import Path
from typing import Optional

import typer
from wasabi import msg

from spacy_llm.util import assemble

Arg = typer.Argument
Opt = typer.Option


def run_pipeline(
    # fmt: off
    text: str = Arg("", help="Text to perform Named Entity Recognition on."),
    config_path: Path = Arg(..., help="Path to the configuration file to use."),
    examples_path: Optional[Path] = Arg(None, help="Path to the examples file to use (few-shot only)."),
    verbose: bool = Opt(False, "--verbose", "-v", help="Show extra information."),
    # fmt: on
):
    msg.text(f"Loading config from {config_path}", show=verbose)
    nlp = assemble(
        config_path,
        overrides={}
        if examples_path is None
        else {"paths.examples": str(examples_path)},
    )
    doc = nlp(text)

    msg.text(f"Text: {doc.text}")
    msg.text(f"Entities: {[(ent.text, ent.label_) for ent in doc.ents]}")


if __name__ == "__main__":
    typer.run(run_pipeline)
```

### create yaml file
```yaml
- text: Jack and Jill went up the hill.
  entities:
    PERSON:
      - Jack
      - Jill
    LOCATION:
      - hill
- text: Jack fell down and broke his crown.
  entities:
    PERSON:
      - Jack
- text: Jill came tumbling after.
  entities:
    PERSON:
      - Jill
```
